# Projeto 1 - Conexão entre Bucket S3 e banco de dados Postgres no RDS
Este projeto consiste em ler um bucket S3 com arquivos de imagens, e armazenar os nomes dessas imagens em um banco de dados (RDS).

## Preparando o ambiente

1. Instalar a biblioteca **python-dotenv** para trabalhar com variáveis de ambiente, a fim de proteger dados sensíveis.

In [ ]:
!pip install python-dotenv

2. Importar as variáveis de ambiente do arquivo **.env** no mesmo diretório

In [ ]:
# Importa um método da biblioteca dotenv
from dotenv import load_dotenv
import os

# Carrega as variáveis de ambiente do arquivo .env
load_dotenv()

## Criando o banco de dados padrão, para, a partir dele, começar a trabalhar
Neste caso, é criado o database "postgres", e a partir dele, é possível criar o database "inventário", que é o que iremos trabalhar

3. Importar biblioteca **psycopg2** para possiblitar conexão com bancos de dados Postgres

In [ ]:
!pip install psycopg2

4. Realizar a conexão com o banco de dados Postgres no RDS, e criar um database chamado **"inventário"** dentro dele, onde serão armazenados os arquivos do Bucket S3

In [ ]:
import psycopg2 #lib para possibilitar conexão com algum BD Postgres

db_host = os.environ.get('DB_HOST') # pegando o host (endpoint RDS) das variáveis de ambiente
db_user = os.environ.get('DB_USER') # pegando o usuário do BD RDS das variáveis de ambiente
db_password = os.environ.get('DB_PASSWORD') # pegando a senha do usuário do BD RDS das variáveis de ambiente

conexaoBd = psycopg2.connect(host=db_host, database='postgres',
                             user=db_user, password=db_password)
# host -> endpoint do BD, endereço desse banco
# database -> BD em que se quer conectar, como não existe nenhum criado ainda, utilizou-se o BD default "postgres"
# user -> usuário para acessar o BD (configurado na criação do banco, geralmente é postgres mesmo, pois não tem BD criado ainda)
# password -> senha definida na criação do BD

conexaoBd.autocommit = True # transações commitadas automaticamente

cursor = conexaoBd.cursor() # objeto para realizar consultas no banco de dados, no caso a var cursor será o cursor do conexaoBD, que é a conexão
                            # do BD feita anteriormente

cursor.execute('create database inventario;') # via cursor, está executando um create database para criar o BD inventário

conexaoBd.close() # fecha a conexão, senão ela fica aberta para sempre

## Criando tabela no banco de dados invetário criado anteriormente
Cria a tabela onde serão armazenados os arquivos, com os campos necessários "id do arquivo" e "nome do arquivo".

5. Ao criar a tabela **arquivos**, dentro do banco de dados inventário, é definido quais campos ela terá. No caso, ela terá um campo de inteiros onde terá os identificadores (ids) de cada arquivo e um campo de caracteres (varchar(256)) com o nome do arquivo presente no Bucket S3

In [ ]:
import psycopg2 #lib para possibilitar conexão com algum BD Postgres

conexaoBd = psycopg2.connect(host=db_host, database='inventario',
                             user=db_user, password=db_password)
# database -> BD em que se quer conectar, no caso o criado anteriormente "inventário"

conexaoBd.autocommit = True

cursor = conexaoBd.cursor()

cursor.execute('create table arquivos (idarquivo INT, nomearquivo VARCHAR(256));') # via cursor, está executando um create table
                                                                                # para criar a tabela onde serão armazenadas o ID do arquivo e o nome

conexaoBd.close()

## Conectar ao bucket S3 onde estão os arquivos

6. Instalar a lib **boto3** para comunicar com serviços AWS

In [ ]:
!pip install boto3

7. Conectar com o Bucket S3 e listar os arquivos que estão presentes nele

In [ ]:
import boto3 # lib para comunicar com serviços AWS

s3_region = os.environ.get('S3_REGION') # pegando a região do S3 das variáveis de ambiente
aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID') # pegando a chave de acesso AWS das variáveis de ambiente
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY') # pegando a chave secreta de acesso AWS das variáveis de ambiente

s3 = boto3.resource( # identificando qual serviço da AWS irei utilizar aqui, no caso, o s3, realizando a "conexão" com esse serviço AWS
    service_name = 's3', # nome do serviço = s3
    region_name = s3_region, # região onde meu bucket está
    aws_access_key_id = aws_access_key_id, # chave de acesso AWS criada
    aws_secret_access_key = aws_secret_access_key # chave de acesso AWS privada
)

bucket_name = os.environ.get('BUCKET_NAME') # pegando o nome do bucket das variáveis de ambiente
bucket_prefix = os.environ.get('BUCKET_PREFIX') # pegando o prefixo (pasta) dentro do bucket das variáveis de ambiente

for objects_s3 in s3.Bucket(bucket_name).objects.filter(Prefix=bucket_prefix):
  # objects_s3 -> variável de controle do for | s3.Bucket(bucket) -> identifica o bucket | objects.filter(Prefix=prefix) -> filtra pela pasta definida
  if objects_s3.key.endswith('jpg') or objects_s3.key.endswith('JPG'): # key -> nome do arquivo
    filename = objects_s3.key.split('/')[1]
    print(filename)

8. Gravar no BD Postgres, que está no RDS, os nomes dos arquivos que estão no Bucket S3 com um identificador incremental

In [ ]:
s3 = boto3.resource( # identificando qual serviço da AWS irei utilizar aqui, no caso, o s3, realizando a "conexão" com esse serviço AWS
    service_name = 's3', # nome do serviço = s3
    region_name = s3_region, # região onde meu bucket está
    aws_access_key_id = aws_access_key_id, # chave de acesso AWS criada
    aws_secret_access_key = aws_secret_access_key # chave de acesso AWS privada
)

# CONECTAR COM O BD POSTGRES
conexaoBd = psycopg2.connect(host=db_host, database='inventario', user=db_user, password=db_password)

conexaoBd.autocommit = True

cursor = conexaoBd.cursor()

id = 0 # criando o identificador a ser adicionado na tabela, que será incremental

for objects_s3 in s3.Bucket(bucket_name).objects.filter(Prefix=bucket_prefix):
    if objects_s3.key.endswith('jpg') or objects_s3.key.endswith('JPG'): # key -> nome do arquivo
        filename = objects_s3.key.split('/')[1]
        id += 1
        cursor.execute("insert into arquivos (idarquivo,nomearquivo) values (" + str(id) + ",'" + filename + "')")

conexaoBd.close()

9. Verificar se ocorreu a inserção dos dados no BD Postgres

In [ ]:
conexaoBd = psycopg2.connect(host=db_host, database='inventario', user=db_user, password=db_password)

conexaoBd.autocommit = True

cursor = conexaoBd.cursor() 

cursor.execute('select * from arquivos;')

todosArquivosTabelaBD = cursor.fetchall() # pega todos os itens da tabela arquivos (q foi feito o select antes) e coloca na var todosArquivosTabelaBD

for item in todosArquivosTabelaBD:
    print(item)

conexaoBd.close()